<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/main/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from bs4 import BeautifulSoup as b4
import asyncio
import multiprocessing
import time
import requests
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [59]:
TRIAL = 3

### Parse chapter

In [61]:
def parseFile(html):
  #with open(html, "r") as file:
  soup = b4(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  content = ""
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text()
  soup = soup.find_all('p')
  for p in soup:
    content.append(p.get_text())
  print(content)
  with open(title, "w" ) as file:
    file.write(content)
  file.close()

### Download main webpage

In [62]:
def getChapter(url, browser, trial):
  try:
      browser.get(url)
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
      parseFile(html)
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
      else:
        getChapter(url, browser, trial + 1)
  finally:
      browser.quit()

### Get webpage

In [63]:
def getNovelPage(url, browser, trial):
  try:
      browser.get(url)
      html = browser.page_source
      soup = b4(html, features="html.parser")
      strainer = soup.find(class_="chapter-list").find_all('a')
      for link in strainer:
        # TODO: Make sure to add link plus the href
        getChapter(link['href'], browser, 0)
      return html
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        return -1
        file.close()
        print("I give up...")
      else:
        getNovelPage(url, browser, trial + 1)
  finally:
      browser.quit()

### Get links and go to next chapter page, if avaliable (testing from file)

In [64]:
def getChapterPages(url, page_num):
  # Start a new selenium process
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)

  # Get the novel main page
  url_source = getNovelPage(url)

  if url_source == -1:
    print("Unable to find page source!")
    return

  soup = b4(url_source, features="html.parser")
  links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
  next_url = ""
  for link in links:
    next_url = link.find('a')['href']



  # Make new url
  new_url = url.replace("/chapters", "") + next_url
  getChapterPages(new_url, page_num+1)

### Input url

In [ ]:
url = input("Enter URL: ")
#sample url: https://www.lightnovelpub.com/novel/chrysalis-16091353/chapters
getChapterPages(url, 1)

Enter URL: https://www.lightnovelpub.com/novel/chrysalis-552/chapter-1-30041322


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/124.0.6367.91/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x587c9c522cb3 <unknown>
#1 0x587c9c2114a7 <unknown>
#2 0x587c9c245a26 <unknown>
#3 0x587c9c241980 <unknown>
#4 0x587c9c28a7b0 <unknown>
#5 0x587c9c27e1b3 <unknown>
#6 0x587c9c24f24a <unknown>
#7 0x587c9c24fc1e <unknown>
#8 0x587c9c4e6fdb <unknown>
#9 0x587c9c4eaf2b <unknown>
#10 0x587c9c4d30f1 <unknown>
#11 0x587c9c4eba92 <unknown>
#12 0x587c9c4b7faf <unknown>
#13 0x587c9c511e08 <unknown>
#14 0x587c9c511fe0 <unknown>
#15 0x587c9c521e04 <unknown>
#16 0x792c51cf9ac3 <unknown>
